In [1]:
# Modified script to generate the Halton Sampler code
import sys

# How many components to generate.
num_dimensions = 256

# Check primality. Not optimized, since it's not performance-critical.
def is_prime(p):
    for i in range(2, p):
        if not p % i:
            return False
    return True

# Init prime number array.
primes = []
candidate = 1
for i in range(num_dimensions):
    while (True):
        candidate += 1
        if (is_prime(candidate)):
            break;
    primes.append(candidate)

In [11]:
sizes = []
tsize = 0
for p in primes:
    if p == 2:
        sizes.append(0)
        tsize += 0
    elif p == 3:
        sizes.append(243)
        tsize += 243
    elif p == 5:
        sizes.append(125)
        tsize += 125
    elif p == 7:
        sizes.append(343)
        tsize += 343
    elif p == 11:
        sizes.append(121)
        tsize += 121
    elif p == 13:
        sizes.append(169)
        tsize += 169
    elif p == 17:
        sizes.append(289)
        tsize += 289
    elif p == 19:
        sizes.append(361)
        tsize += 361
    else:
        sizes.append(p)
        tsize += p

print("Total Kb : %d"  % (tsize * 2 / 1024))


Total Kb : 377


In [2]:
# direct C++ -> python code
def invert(base, digits, index, perms):
    result = 0;
    for i in range(0, digits):
        result = result * base + perm[index % base]
        index /= base
    return result

def initTables(perms):
    

def initFaure():
    max_base = primes[-1]
    perms = []
    for k in range(1, 4):
        temp = []
        for i in range(0, k):
            temp.append(i)
            pass
        perms.append(temp)
        pass
    for base in range(4, max_base + 1):
        b = base / 2
        temp = []
        if base & 1:
            for i in range(0, base - 1):
                pass
        else:
            for i in range(0, b):
                




IndentationError: expected an indented block (<ipython-input-2-17692dcf84e2>, line 12)

In [9]:
print('''
static const float theInv2p32 = 1.0f / (uint64(1) << 32);

// Compute points of the Halton sequence with with digit-permutations for different bases.
class BRAY_Halton
{
public:
    // Initialize permutation tables using Faure permutations
    static void initFaure();
    
    // Precompute the inversion tables
    static uint16 invert(uint16 base, uint16 digits, uint index, const UT_Array<uint16>& perm);

    // Precompute faure permute tables
    static void initTables(const UT_Array<UT_Array<uint16>>& perms);
    
    // Return the number of supported dimensions.
    static uint getDimensions() { return %du; }

    // Return the Halton sample for the given dimension (component) and index.
    // If sample is called with dim > maxDimensions, then uniform random numbers are generated
    static float sample(uint dim, uint index);
''' % num_dimensions)

for i in range(0, num_dimensions):
    print('    static float halton%d(uint index);' % primes[i])

# The following strings will be extended below.
perm_arrays = '' # Permutation arrays.
init_tables = '' # Loops for initializing the permutation arrays.

# Individual implementations for each dimensions.
halton_impl = '''
// Special case: radical inverse in base 2, with direct bit reversal.
float
BRAY_Halton::halton2(uint index)
{
    index = (index << 16) | (index >> 16);
    index = ((index & 0x00ff00ff) << 8) | ((index & 0xff00ff00) >> 8);
    index = ((index & 0x0f0f0f0f) << 4) | ((index & 0xf0f0f0f0) >> 4);
    index = ((index & 0x33333333) << 2) | ((index & 0xcccccccc) >> 2);
    index = ((index & 0x55555555) << 1) | ((index & 0xaaaaaaaa) >> 1);
    return index * theInv2p32;
}
'''

for i in range(1, num_dimensions): # Skip base 2.
    base = primes[i]

    # Based on the permutation table size, we process multiple digits at once.
    digits = 1
    pow_base = base
    while pow_base * base <= 1: # Maximum permutation table size.
        pow_base *= base
        digits += 1
        
    print(iter)

    perm_arrays += '    static uint16 thePerm%d[%d];\n' % (base, pow_base)

    max_power = pow_base
    while max_power * pow_base < (1 << 32): # 32-bit unsigned precision
        max_power *= pow_base
        
    print(pow_base, max_power)
    power = max_power / pow_base

    halton_impl += '''
float
BRAY_Halton::halton%d(uint index)
{
    return (thePerm%d[index %% %du] * %du +
    ''' % (base, base, pow_base, power)

    init_tables += '''
    for (unsigned short i = 0; i < %d; ++i)
       thePerm%d[i] = invert(%d, %d, i, perms[%d]);''' % (pow_base, base, base, digits, base)

    # Advance to next set of digits.
    div = 1
    while power / pow_base > 1:
        div *= pow_base
        power /= pow_base
        halton_impl += '            thePerm%d[(index / %du) %% %du] * %du +\n' % (base, div, pow_base, power)

    halton_impl += '''            thePerm%d[(index / %du) %% %du])
            * float(0x1.fffffcp-1 / %du); // Results in [0,1).
}
''' % (base, div * pow_base, pow_base, max_power)

# end the class
print('\n' + perm_arrays + '};')


print('''
void
BRAY_Halton::initFaure()
{
    const uint max_base = %du;
    UT_Array<UT_Array<uint16>> perms(max_base + 1, max_base + 1);
     // Keep identity permutations for base 1, 2, 3.
    for (uint k = 1; k <= 3; ++k)
    {
        perms[k].setSize(k);
        for (unsigned i = 0; i < k; ++i)
            perms[k][i] = i;
    }
    for (uint base = 4; base <= max_base; ++base)
    {
        perms[base].setSize(base);
        const unsigned b = base / 2;
        if (base & 1) // odd
        {
            for (unsigned i = 0; i < base - 1; ++i)
                perms[base][i + (i >= b)] = perms[base - 1][i] + (perms[base - 1][i] >= b);
            perms[base][b] = b;
        }
        else // even
        {
            for (unsigned i = 0; i < b; ++i)
            {
                perms[base][i] = 2 * perms[b][i];
                perms[base][b + i] = 2 * perms[b][i] + 1;
            }
        }
    }
    initTables(perms);
}

float
BRAY_Halton::sample(uint dimension, uint index)
{
    switch (dimension)
    {''' % (primes[-1]))

for i in range(num_dimensions):
    print('''        case %d:
            return halton%d(index);''' % (i, primes[i]))
print('''    }
    return 0.f;
}

uint16
BRAY_Halton::invert(uint16 base, uint16 digits, uint index, const UT_Array<uint16>& perm)
{
    uint16 result = 0;
    for (uint16 i = 0; i < digits; ++i)
    { 
        result = result * base + perm[index % base];
        index /= base;
    }
    return result;
}

void
BRAY_Halton::initTables(const UT_Array<UT_Array<uint16>>& perms)
{''')

print(init_tables + '\n}')

print(halton_impl)


static const float theInv2p32 = 1.0f / (uint64(1) << 32);

// Compute points of the Halton sequence with with digit-permutations for different bases.
class BRAY_Halton
{
public:
    // Initialize permutation tables using Faure permutations
    static void initFaure();
    
    // Precompute the inversion tables
    static uint16 invert(uint16 base, uint16 digits, uint index, const UT_Array<uint16>& perm);

    // Precompute faure permute tables
    static void initTables(const UT_Array<UT_Array<uint16>>& perms);
    
    // Return the number of supported dimensions.
    static uint getDimensions() { return 2u; }

    // Return the Halton sample for the given dimension (component) and index.
    // If sample is called with dim > maxDimensions, then uniform random numbers are generated
    static float sample(uint dim, uint index);

    static float halton2(uint index);
    static float halton3(uint index);
5
3 3486784401

    static uint16 thePerm3[3];
};

void
BRAY_Halton::initFaure()